In [7]:
#Code to create a company brochure based on their website and its links using multi shot prompting

In [2]:
#Translate the given brochure into Kannada and display

In [3]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [5]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key?")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [6]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [8]:
link_system_prompt = """You are provided with a list of links found on a webpage. 
Your task is to determine which of these links are most relevant to include in a company brochure. 
Relevant links include those pointing to an About page, a Company page, or Careers/Jobs pages.

You should respond in JSON format, as shown in the following examples:

Example 1:
Input Links:
[
    "https://example.com/home",
    "https://example.com/about",
    "https://example.com/contact",
    "https://example.com/careers"
]

Response:
{
    "links": [
        {"type": "about page", "url": "https://example.com/about"},
        {"type": "careers page", "url": "https://example.com/careers"}
    ]
}

Example 2:
Input Links:
[
    "https://company.com/index",
    "https://company.com/our-story",
    "https://company.com/team",
    "https://company.com/jobs"
]

Response:
{
    "links": [
        {"type": "about page", "url": "https://company.com/our-story"},
        {"type": "careers page", "url": "https://company.com/jobs"}
    ]
}

Now, given a new list of links, provide a JSON response in the same format.
"""


In [9]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [12]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [13]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [14]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    #display(Markdown(result))
    return result

In [15]:
system_prompt2="You are a translator who translates company brochures originally in english to Kannada. \
maintain the exact meaning and formatting of the input brochure where the input brochure will be given in markdown.\
give the translated text in markdown as well"

In [16]:
def translate_user_prompt(company_name,url):
    brochure = create_brochure(company_name,url)
    user_prompt="You are given a brochure in markdown format in english as below + \n"
    user_prompt+=brochure
    user_prompt+="Translate the entire brochure to Kannada and reply in markdown"
    return user_prompt
    

In [18]:
def stream_translated_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": translate_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [19]:
#example output
stream_translated_brochure("HuggingFace","https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}


# ಹಗಿಂಗ್ ಫೇಸ್ ಬ್ರೋಶರ್

### ಭವಿಷ್ಯವನ್ನು ರೂಪಿಸುತ್ತಿರುವ ಎಐ ಸಮುದಾಯ

ಹಗಿಂಗ್ ಫೇಸ್ ಒಂದು ಪ್ರಮುಖ ವೇದಿಕೆಯಾಗಿದ್ದು, ಯಾಂತ್ರಿಕ ಕಲಿಕೆ ಸಮುದಾಯವನ್ನು ಒಟ್ಟುಗೂಡಿಸುತ್ತದೆ, ಮಾದರಿಗಳು, ಡೇಟಾಸೆಟ್‌ಗಳು ಮತ್ತು ಅಪ್ಲಿಕೇಶನ್‌ಗಳ ಮೇಲೆ ಸಹಕಾರವನ್ನು ಉತ್ತೇಜಿಸುತ್ತದೆ. 10% ವೈದ್ಯಕೀಯ ಸಂಪತ್ತು ನುರಿತ ಸಮುದಾಯವು AI ಸಹಾಯದಿಂದ ಭಾರತದಲ್ಲಿ ತಂತ್ರಜ್ಞಾನವನ್ನು ಮತ್ತು ತಂತ್ರಾಂತರವನ್ನು ಉತ್ತೇಜಿಸ್ತದೆ. 

---

## ನಮ್ಮ ಬಗ್ಗೆ

ಹಗಿಂಗ್ ಫೇಸ್‌ನಲ್ಲಿ, ಸಮುದಾಯದ ಶಕ್ತಿಯಲ್ಲಿ ನಮಗೆ ನಂಬಿಕೆ ಇದೆ. ಪ್ರತಿಯೊಬ್ಬರಿಗೂ ಉನ್ನತ ಯಾಂತ್ರಿಕ ಕಲಿಕೆ ಸಾಧನಗಳನ್ನು ಲಭ್ಯವಾಗಿಸುವ ಮೂಲಕ, ನಾವು ಒಟ್ಟಾಗಿ ಭವಿಷ್ಯವನ್ನು ನಿರ್ಮಿಸುತ್ತೇವೆ. ನಮ್ಮ ಓಪನ್-ಮೂಲ ವೇದಿಕೆ ಅತ್ಯುತ್ತಮ ಮಾದರಿಗಳು ಮತ್ತು ಡೇಟಾಸೆಟ್‌ಗಳ ಅಭಿವೃದ್ಧಿಯನ್ನು ಬೆಂಬಲಿಸುತ್ತದೆ, ನಾವೆಲ್ಲಾ ಕಾರ್ಯಕ್ಷಮತೆ ಮತ್ತು ಏಕಕಾಲದ ಬಳಕೆಯ ಅವಕಾಶವನ್ನು ಪದೇ ಪದೇ ಪಡೆದುಕೊಳ್ಳುತ್ತೇವೆ.

---

## ನಾವು ಏನು ನೀಡುತ್ತೇವೆ

- **ಮಾದರಿಗಳು**: ಪಠ್ಯದಿಂದ ಚಿತ್ರ ಪ್ರಕ್ರಿಯೆ, ವಿಡಿಯೋ ವಿಶ್ಲೇಷಣೆ ಮತ್ತು ಇನ್ನಷ್ಟು ವಿಭಾಗಗಳಿಗೆ 10% ವೈದ್ಯಕೀಯ ಸಂಪತ್ತು ನುರಿತ ಸಮುದಾಯವು AI ಸಂಪತ್ತು ವ್ಯಾಪಾರವನ್ನು ಪುನರಾವೃತ್ತ ಓಪನ್-ಮೂಲ ಪಠ್ಯ ಲೇಖನ (LE) ಬಳಸಲಾಗುತ್ತದೆ.

- **ಡೇಟಾಸೆಟ್‌ಗಳು**: ನಿಮ್ಮ ಪ್ರಾಜೆಕ್ಟ್‌ಗಳಿಗೆ ತಾಂಗುವ ಮೂಲಕ, 250,000ಕ್ಕೂ ಹೆಚ್ಚು ಸಾರ್ವಜನಿಕ ಡೇಟಾಪತ್ರಗಳು ಹಂಚಿಕೊಳ್ಳುವ ಸಜ್ಜಗಣದಾಸೆಯಿಂದ ಬೆಂಬಲಿಸುತ್ತವೆ.

- **ಸ್ಪೇಸ್‌ಗಳು**: ನಿಮ್ಮ ಅಪ್ಲಿಕೇಶನ್‌ಗಳನ್ನು ಸುಲಭವಾಗಿ ನಿರ್ವಹಿಸಲು ಆಯ್ಕೆ ಮಾಡಬಹುದು.

- **ಉದ್ಯಮ ಪರಿಹಾರಗಳು**: ನಮ್ಮ ಉದ್ಯಮ-ಮಟ್ಟದ ವ್ಯವಹಾರಗಳು ಉತ್ತಮ ಭದ್ರತೆ, ವಿಶೇಷ ಬೆಂಬಲವನ್ನು ನೀಡುತ್ತವೆ.

- **ಕಂಪ್ಯೂಟ್**: ನಿರ್ವಹಿತ ಇಂಜಿನಿಯರಿಂಗ್ ಏಷಾಕ್‌ಗಳನ್ನು $0.60/ಗಂಟೆ ಪ್ರಾರಂಭಿಸುತ್ತವೆ.

---

## ನಮ್ಮ ಗ್ರಾಹಕರು

ಹಗಿಂಗ್ ಫೇಸ್ 50,000ಕ್ಕೂ ಹೆಚ್ಚು ಸಂಸ್ಥೆಗಳಿಗೆ ಉಲ್ಲೇಖಿಸುತ್ತಿದೆ, ಇದರಲ್ಲಿ ಉತ್ತಮವು:

- **ಮೆಟಾವರ್ಸ್ ನಾಯಕರಿಗೆ**: ಮೆಟಾದ ಎಐ.
- **ಕರ್ನಾಟಕದ ಕಂಬನಿಗಳ**: ಅಫೆಸ್ಟ್ ವಾದದ ಬೆವ್ವಾನ್, Google.
- **ತಂತ್ರಜ್ಞಾನಗಳ ಮಧ್ಯಸ್ಥಿಕೆ**: Microsoft, Intel.
- **ಸೃಜನಾತ್ಮಕ ಪರಿಹಾರಗಳು**: Grammarly, Writer.

ನಮ್ಮ ವೇದಿಕೆಯನ್ನು ಸ್ವಾಯತ್ತ ಸಂಶೋಧಕರಿಂದ ಹಿಡಿದು, ದೊಡ್ಡ ಕಂಪನಿಯ ತಂತ್ರಜ್ಞಾನವನ್ನು ಬೆಂಬಲಿಸುವಂತೆ ರೂಪಿಸಲಾಗಿದೆ.

---

## ಕಂಪನಿಯ ಸಾಂಸ್ಕೃತಿಕ

ಹಗಿಂಗ್ ಫೇಸ್‌ನಲ್ಲಿ, ನಾವು ಒಳಕೋಡು, ಸಹಯೋಗ ಮತ್ತು ನಾವೀನ್ಯತೆಗಳ ಸಾಂಸ್ಕೃತಿಕವನ್ನು ಉತ್ತೇಜಿಸುತ್ತೇವೆ. ನಮ್ಮ ತಂಡದ ಸದಸ್ಯರು ತಮ್ಮ ಆಲೋಚನೆಗಳನ್ನು ಮತ್ತು ಪರಿಣತಿಯನ್ನು ಹಂಚಿಕೊಳ್ಳಲು ನಾವು ಉತ್ಸಾಹಿಸುತ್ತೇವೆ, ಇದು AI ಕ್ಷೇತ್ರವನ್ನು ಮುನ್ನಡೆಸುವಲ್ಲಿಗೆ ಒಡ್ಡುವ ಚಟುವಟಿಕೆಯನ್ನು ಮತ್ತು ಪ್ರಭೇಕಾರಿಯಾಗಿ ಹೊಡೆಯುತ್ತಿದೆ. ನಮ್ಮ ಆಳವಾದ ತತ್ವವನ್ನು ಈ ಒಳಹೊರೆಯಲಾಗುತ್ತದೆ, ಇದು ವಿಶೇಷ ರೀತಿಯಲ್ಲಿನ ಪ್ರಾಬಲ್ಯ ಮತ್ತು ಸಾಮಾಜಿಕ ಶಿಕ್ಷಣವನ್ನು ಉತ್ತೇಜಿಸುತ್ತದೆ, ತಂತ್ರಜ್ಞಾನ ಮತ್ತು AI ಗೆ ಆಸಕ್ತಿ ಹೊಂದಿರುವ ವ್ಯಕ್ತಿಗಳಿಗಾಗಿ ವಿಸ್ತೃತಕ್ಕೆ ಉತ್ತಮ ಸ್ಥಳವಾಗಿದೆ.

---

## ಉದ್ಯೋಗಾವಕಾಶಗಳು

ನಮ್ಮ ಸಮಗ್ರವಾದ ತಂಡದಲ್ಲಿ ಸೇರಿ AI ನ ಭವಿಷ್ಯವನ್ನು ರೂಪಿಸವರಿಗೆ ಸಹಾಯ ಮಾಡು. software development, research, and support ನಲ್ಲಿ ಶ್ರೇಷ್ಠ ಕೊಡುಗೆಗೆ ಬಲವಾದ ವ್ಯಕ್ತಿಗಳನ್ನು ಹುಡುಕುತ್ತಿದ್ದೇವೆ.  ಹಗಿಂಗ್ ಫೇಸ್‌ನಲ್ಲಿ, ನೀವು ನಿಮ್ಮ ಕೌಶಲ್ಯಗಳನ್ನು ವೃದ್ಧಿಸಲು, ಪ್ರಮುಖ ಯೋಜನೆಗಳಲ್ಲಿ ಸಹಯೋಗಿಸಲು, ಮತ್ತು ಕ人工 inteligencia ಭಾರತದಲ್ಲಿ ಶ್ರೇಷ್ಟವಾದರೂ ವಿಷಯವನ್ನು ರೂಪಿಸಲು ಅವಕಾಶ ಪಡೆಯುತ್ತೀರಿ.

ಉದ್ಯೋಗಾವಕಾಶಗಳ ಬಗ್ಗೆ ಹೆಚ್ಚಿನ ಮಾಹಿತಿಗೆ, ದಯವಿಟ್ಟು ನಮ್ಮ [ಜಾಬ್ಸ್ ಪುಟವನ್ನು](https://huggingface.co/jobs) ನೋಡಿ.

---

### ನಮ್ಮೊಂದಿಗೆ ಸಂಪರ್ಕಿಸಿ

ನವೀನ ವಿಷಯಗಳನ್ನು ಪಡೆಯಲು ಮತ್ತು ನಮ್ಮ ಸಾಮಾಜಿಕ ಚಾನಲ್‌ಗಳನ್ನು ಅನುಸರಿಸಿ:

- [ಟ್ವಿಟ್ಟರ್](https://twitter.com/huggingface)
- [ಲಿಂಕ್ಡ್‌ಇನ್](https://www.linkedin.com/company/huggingface)
- [ಗಿಥಬ್](https://github.com/huggingface)
- [ಡಿಸ್ಕೋರ್ಡ್](https://discord.gg/huggingface)

AI ಯ ಅಪಾಯ ವಿಭಾಗವನ್ನು ಅಭಿವೃದ್ಧಿ ಮತ್ತು ಈ ಪುಟವನ್ನು ಹೆಚ್ಚು ತಿಳಿಯಿರಿ [ಹಗಿಂಗ್ ಫೇಸ್](https://huggingface.co) ನಲ್ಲಿ.

---

AI ಯ ಪುನಾರ್ಘರದ ನಿರ್ಮಾಣ ಮತ್ತು ಭವಿಷ್ಯದ ಮಾದರಿಗಳಿಗೆ ಪರಿಣಾಮವನ್ನು ರೂಪಿಸಲು ನಮ್ಮೊಂದಿಗೆ ಒಟ್ಟಾಗಿ ಸೇರಿ!